In [20]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import scale
from sklearn import metrics

In [21]:
data = pd.read_excel('./일일 기술데이터.xlsx')

KeyboardInterrupt: 

In [ ]:
# 회사별로 reshape  20
def reshape_fin_dataset(data):
    ##  쓸데없는 칼럼 제거
    company_name = list((data.iloc[8]))
    col_name = list(data.iloc[11])
    data2 = data.iloc[13:]
    data2.columns = col_name
    col_num = len(list(set(col_name)))
    dummy = pd.DataFrame([[np.nan]])
    # 더미 데이터셋 생성
    sets = pd.DataFrame()    
    for i in range(len(set(company_name))):
        ## 회사 별로 뽑아오기
        sub_dataset = data2.iloc[:,i*col_num : (i+1)*col_num][:-1]
        sub_dataset['Name'] = company_name[i*col_num]
        sub_rev = sub_dataset['수익률(%)'][1:]
        sub_rev2 = sub_rev.append(dummy)
        sub_rev2.index = sub_dataset.index
        sub_dataset['수익률(%)'] = sub_rev2
        sets = sets.append(sub_dataset[:-1])
    
    sets2 = sets[['종가 (5일 평균)(원)', '변동성 (5일)', '순매수수량(기관/외국인계)(20일합산)(주)', 
     '순매수수량(외국인계)(20일합산)(주)', 'PER(IFRS-연결)', '순매수수량(개인)(20일합산)(주)', '종가 (60일 평균)(원)', 
     '거래량 (20일 평균)(주)', '순매수수량(외국인계)(60일합산)(주)', '순매수수량(기관/외국인계)(60일합산)(주)', '순매수수량(외국인계)(주)',
     '순매수수량(기관계)(주)', '순매수수량(기관계)(60일합산)(주)', '순매수수량(개인)(주)', '외국인보유비중(티커)(%)', 'PSR(IFRS-연결)',
     '종가 (120일 평균)(원)', '종가 (20일 평균)(원)', '순매수수량(개인)(60일합산)(주)', '거래량 (60일 평균)(주)', 'PCR(IFRS-연결)',
     '거래정지구분', '순매수수량(기관계)(20일합산)(주)', '수익률 (1주)(%)', 'PBR(IFRS-연결)', '거래량 (5일 평균)(주)',
     '순매수수량(기관/외국인계)(주)','수익률(%)', 'Name']]
    sets2['수익률(%)'] = sets2['수익률(%)'].astype('float64')
    return sets2

In [ ]:
# 2013년 9월 ~ 2018 년 6월 완료
tech = reshape_fin_dataset(data)
tech

In [ ]:
coway = tech[tech.Name == '코웨이']

In [ ]:
coway

In [ ]:
tech.isnull().sum()

In [ ]:
# 코웨이 NA 없음
coway.isnull().sum()

In [ ]:
# number 전일 보다 크면 1 아니면 0
def making_direction_of_change(sub_dataset,col,plus,number=1):

    for colname in col:
        # 변화율 백터 생성
        a = sub_dataset[colname]
        g = pd.DataFrame(np.zeros(len(sub_dataset)))
        g.replace(0,np.nan,inplace=True)

        for i in list(range(len(a)))[number:]:
            if list(a)[i] != np.nan and  list(a)[i-number] != np.nan:
                g.loc[i] = a[i] > a[i-number]

            else:
                pass

        g.index = sub_dataset.index
        sub_dataset[colname+plus] = g.astype('float64')
        
    return sub_dataset

In [ ]:
def make_cate_t(data):
    ## 수익률 범주화
    bb = np.array([-100,0.000000001,100])

    Y_data_cat = np.digitize(data['수익률(%)'],bb)
    Y_data_cat = Y_data_cat - 1
    
    return Y_data_cat

In [ ]:
## 증가율로 바꾸기 / null값 지우기 등.. 실행
def preprocessing_data(data):    
    col_name = data.columns
    company_name = data.Name
    scaled_total_dataset = pd.DataFrame()

    for name in list(set(company_name)):
        
        ## 회사 별로 뽑아오기
        sub_dataset = data[data.Name == name]
        
        #거래정지구분에서 null 값있는 row 제거(상장 전 기간 제거)
        select_notNan = sub_dataset['거래정지구분'].dropna()
        sub_dataset = sub_dataset.loc[select_notNan.index]
        
        # 거래정지 로우 & 칼럼 제거
        sub_dataset = sub_dataset[sub_dataset['거래정지구분'] !='거래정지']
        sub_dataset.drop(['거래정지구분'],inplace=True,axis=1)
        
#         #결측치 처리 구간 ***********************************************************************
#         per, psr, pbr => null은 칼럼 평균으로 대체 
#         sub_dataset['PER(IFRS-연결)'].fillna(sub_dataset['PER(IFRS-연결)'].mean(),inplace = True)
#         sub_dataset['PSR(IFRS-연결)'].fillna(sub_dataset['PSR(IFRS-연결)'].mean(),inplace = True)
#         sub_dataset['PBR(IFRS-연결)'].fillna(sub_dataset['PBR(IFRS-연결)'].mean(),inplace = True)
        
        # 변화율로 만들기
        sub_dataset = making_direction_of_change(sub_dataset,['종가 (5일 평균)(원)', '변동성 (5일)',
       '순매수수량(외국인계)(20일합산)(주)', 'PER(IFRS-연결)', '순매수수량(개인)(20일합산)(주)',
       '종가 (60일 평균)(원)', '거래량 (20일 평균)(주)', '순매수수량(외국인계)(60일합산)(주)',
       '순매수수량(기관/외국인계)(60일합산)(주)', '순매수수량(외국인계)(주)', '순매수수량(기관계)(주)',
       '순매수수량(기관계)(60일합산)(주)', '순매수수량(개인)(주)', '외국인보유비중(티커)(%)',
       'PSR(IFRS-연결)', '종가 (120일 평균)(원)', '종가 (20일 평균)(원)','순매수수량(개인)(60일합산)(주)', 
        '거래량 (60일 평균)(주)', 'PCR(IFRS-연결)', '순매수수량(기관/외국인계)(20일합산)(주)',
       '순매수수량(기관계)(20일합산)(주)', '수익률 (1주)(%)', 'PBR(IFRS-연결)', '거래량 (5일 평균)(주)',
       '순매수수량(기관/외국인계)(주)'], '_1일변화율')
        
        sub_dataset = making_direction_of_change(sub_dataset,['종가 (5일 평균)(원)', '변동성 (5일)',
       '순매수수량(외국인계)(20일합산)(주)', 'PER(IFRS-연결)', '순매수수량(개인)(20일합산)(주)',
       '종가 (60일 평균)(원)', '거래량 (20일 평균)(주)', '순매수수량(외국인계)(60일합산)(주)',
       '순매수수량(기관/외국인계)(60일합산)(주)', '순매수수량(외국인계)(주)', '순매수수량(기관계)(주)',
       '순매수수량(기관계)(60일합산)(주)', '순매수수량(개인)(주)', '외국인보유비중(티커)(%)',
       'PSR(IFRS-연결)', '종가 (120일 평균)(원)', '종가 (20일 평균)(원)','순매수수량(개인)(60일합산)(주)', 
        '거래량 (60일 평균)(주)', 'PCR(IFRS-연결)', '순매수수량(기관/외국인계)(20일합산)(주)',
       '순매수수량(기관계)(20일합산)(주)', '수익률 (1주)(%)', 'PBR(IFRS-연결)', '거래량 (5일 평균)(주)',
       '순매수수량(기관/외국인계)(주)'], '_2일변화율',2)

        
        # 수익률 없는 칼럼 삭제
        sub_dataset = sub_dataset[sub_dataset['수익률(%)'].isna() == False]
        
            # inf 처리
        for i in sub_dataset.columns:
            sub_dataset = sub_dataset.loc[sub_dataset[i] != np.inf]
            
        ### 이상치 처리 ## ************************************************************************************        
#         sub_dataset = std_based_outlier(sub_dataset)
        
#        #스케일링
#         sub_dataset = scaling_data(sub_dataset)
        
        scaled_total_dataset = scaled_total_dataset.append(sub_dataset)

    ### 수익률 변환
    scaled_total_dataset['수익률(%)(2)'] = make_cate_t(scaled_total_dataset) 


    scaled_total_dataset.dropna(axis=0,inplace=True)
    return scaled_total_dataset

In [ ]:
dataset = preprocessing_data(coway)

In [ ]:
# 기존 데이터는 일단 남겨두었습니다.
dataset

In [ ]:
dataset.to_excel('./true.xlsx',encoding='utf-8')